In [ ]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
from datetime import datetime

In [ ]:
!pip install pandas

In [ ]:
data = pd.read_csv('2023-2024-Horana-cleaned.csv') 

data['DATE'] = pd.to_datetime(data['DATE'])

data['NetAmount'] = data['NetAmount'].abs()

data['SalesPersonCode']
data.head()

In [ ]:
last_year = data['DATE'].dt.year.max()

# Filter the data to include only the last year
last_year_data = data[data['DATE'].dt.year == last_year]

valid_sales_persons = last_year_data['SalesPersonCode'].unique()

filtered_data = data[data['SalesPersonCode'].isin(valid_sales_persons)]

filtered_data.to_csv('filtered_Horana_cleaned.csv', index=False)

filtered_data.head()

In [ ]:
!pip uninstall tensorflow
!pip install tensorflow==2.4.0

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
!pip install shap

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import shap

In [ ]:
# import numpy as np
# def dummy_npwarn_decorator_factory():
#   def npwarn_decorator(x):
#     return x
#   return npwarn_decorator
# np._no_nep50_warning = getattr(np, '_no_nep50_warning', dummy_npwarn_decorator_factory)

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense
# import shap

In [ ]:
# !pip install numpy 2.0.4?

In [ ]:
# !pip install numpy==2.?1


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


In [ ]:
# !pip install shap
# import shap

In [ ]:
data = pd.read_csv('filtered_Horana_cleaned.csv')

data['DATE'] = pd.to_datetime(data['DATE'])

daily_sales = data.groupby(['DATE', 'SalesPersonCode'])['NetAmount'].sum().reset_index()

pivot_data = daily_sales.pivot(index='DATE', columns='SalesPersonCode', values='NetAmount').fillna(0)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(pivot_data)

def create_dataset(data, time_step=30):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), :])
        y.append(data[i + time_step, :])
    return np.array(X), np.array(y)

time_step = 30  
X, y = create_dataset(scaled_data, time_step)

X = X.reshape(X.shape[0], X.shape[1], X.shape[2])

In [ ]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(time_step, X.shape[2])))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(X.shape[2]))  

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X, y, epochs=20, batch_size=32, verbose=1)

In [ ]:
last_30_days = scaled_data[-time_step:]  
predictions = []

for _ in range(30):  
    last_30_days_reshaped = last_30_days.reshape(1, time_step, X.shape[2])
    next_day_pred = model.predict(last_30_days_reshaped)
    predictions.append(next_day_pred[0])
    last_30_days = np.vstack([last_30_days[1:], next_day_pred])

predictions = scaler.inverse_transform(predictions)

prediction_dates = pd.date_range(start=pivot_data.index[-1] + pd.Timedelta(days=1), periods=30)
prediction_df = pd.DataFrame(predictions, columns=pivot_data.columns, index=prediction_dates)

prediction_df.to_csv('upcoming_month_sales_predictions.csv')

In [ ]:
def predict_upcoming_month_sales(sales_person_code):
    last_30_days = scaled_data[-time_step:]  
    predictions = []

    for _ in range(30): 
        last_30_days_reshaped = last_30_days.reshape(1, time_step, X.shape[2])
        next_day_pred = model.predict(last_30_days_reshaped)
        predictions.append(next_day_pred[0])
        last_30_days = np.vstack([last_30_days[1:], next_day_pred])

    predictions = scaler.inverse_transform(predictions)

    prediction_dates = pd.date_range(start=pivot_data.index[-1] + pd.Timedelta(days=1), periods=30)
    prediction_df = pd.DataFrame(predictions, columns=pivot_data.columns, index=prediction_dates)

    total_sales = prediction_df[sales_person_code].sum()
    return total_sales

In [ ]:
sales_person_code = '265'
total_predicted_sales = predict_upcoming_month_sales(sales_person_code)
print(f"Total predicted sales for SalesPersonCode {sales_person_code} in the upcoming month: {total_predicted_sales:.2f}")

In [ ]:
train_size = int(len(scaled_data) * 0.8)
train_data, test_data = scaled_data[:train_size], scaled_data[train_size - time_step:]


In [ ]:
X_test, y_test = create_dataset(test_data, time_step)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2])


In [ ]:
y_pred_scaled = model.predict(X_test)


In [ ]:
y_pred = scaler.inverse_transform(y_pred_scaled)
y_test_original = scaler.inverse_transform(y_test)


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

mse = mean_squared_error(y_test_original, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_original, y_pred)

print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller

data = pd.read_csv('2023-2024-Horana-cleaned.csv')
data['DATE'] = pd.to_datetime(data['DATE'])
data['NetAmount'] = data['NetAmount'].abs()

sales_265 = data[data['SalesPersonCode'] == '0']

sales_265_monthly = sales_265.resample('M', on='DATE')['NetAmount'].sum().reset_index()

plt.figure(figsize=(12, 6))
plt.plot(sales_265_monthly['DATE'], sales_265_monthly['NetAmount'], marker='o', linestyle='-')
plt.title('Monthly Sales Trend for SalesPersonCode 254')
plt.xlabel('Date')
plt.ylabel('Total Sales')
plt.grid(True)
plt.show()

In [ ]:
def adf_test(series):
    result = adfuller(series, autolag='AIC')
    print("ADF Test Results:")
    print(f"Test Statistic: {result[0]}")
    print(f"P-Value: {result[1]}")
    print(f"Critical Values: {result[4]}")
    if result[1] <= 0.05:
        print("Data is stationary (reject H0)")
    else:
        print("Data is non-stationary (fail to reject H0)")

adf_test(sales_265_monthly['NetAmount'])


In [ ]:
sales_265_monthly['Rolling_Mean'] = sales_265_monthly['NetAmount'].rolling(window=3).mean()
sales_265_monthly['Rolling_Std'] = sales_265_monthly['NetAmount'].rolling(window=3).std()

plt.figure(figsize=(12, 6))
plt.plot(sales_265_monthly['DATE'], sales_265_monthly['NetAmount'], label='Original Sales', color='blue')
plt.plot(sales_265_monthly['DATE'], sales_265_monthly['Rolling_Mean'], label='Rolling Mean', color='red')
plt.plot(sales_265_monthly['DATE'], sales_265_monthly['Rolling_Std'], label='Rolling Std Dev', color='green')
plt.title('Rolling Mean & Standard Deviation for SalesPersonCode 254')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
sales_265_monthly['Diff_1'] = sales_265_monthly['NetAmount'].diff().dropna()

print("\nADF Test after First Differencing:")
adf_test(sales_265_monthly['Diff_1'].dropna())

plt.figure(figsize=(12, 6))
plt.plot(sales_265_monthly['DATE'], sales_265_monthly['Diff_1'], marker='o', linestyle='-')
plt.title('Differenced Monthly Sales for SalesPersonCode 265')
plt.xlabel('Date')
plt.ylabel('Differenced Sales')
plt.grid(True)
plt.show()


In [ ]:
pip install --upgrade shap tensorflow keras numpy


In [ ]:
!pip install seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

data = pd.read_csv('2023-2024-Horana-cleaned.csv')
data['DATE'] = pd.to_datetime(data['DATE'])
data['NetAmount'] = data['NetAmount'].abs()  # Ensure all values are positive

sales_265 = data[data['SalesPersonCode'] == '265']

sales_265_monthly = sales_265.resample('M', on='DATE')['NetAmount'].sum().reset_index()
sales_265_monthly.set_index('DATE', inplace=True)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_sales = scaler.fit_transform(sales_265_monthly[['NetAmount']])

sales_265_monthly['NetAmount_Diff'] = sales_265_monthly['NetAmount'].diff().dropna()
sales_265_monthly.dropna(inplace=True)

scaler_diff = MinMaxScaler(feature_range=(-1, 1))
scaled_diff_data = scaler_diff.fit_transform(sales_265_monthly[['NetAmount_Diff']])

def create_dataset(data, time_step=12):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:i + time_step, 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

time_step = 12  
X, y = create_dataset(scaled_diff_data, time_step)

X = X.reshape(X.shape[0], X.shape[1], 1)

train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(time_step, 1)),
    LSTM(50, return_sequences=False),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, y_train, epochs=50, batch_size=8, verbose=1)

last_12_months = scaled_diff_data[-time_step:].reshape(1, time_step, 1)
predicted_scaled_diff = model.predict(last_12_months)

predicted_diff_value = scaler_diff.inverse_transform([[predicted_scaled_diff[0][0]]])[0][0]
previous_month_sales = sales_265_monthly['NetAmount'].iloc[-1]
predicted_sales = previous_month_sales + predicted_diff_value  # Reverse differencing

actual_sales = sales_265_monthly['NetAmount'].iloc[-1]

print(f"Actual Sales for Last Month: {actual_sales:.2f}")
print(f"Predicted Sales for Last Month: {predicted_sales:.2f}")

plt.figure(figsize=(10, 5))
plt.plot(sales_265_monthly.index[-12:], sales_265_monthly['NetAmount'][-12:], label="Actual", marker='o')
plt.axhline(y=predicted_sales, color='r', linestyle='--', label="Predicted Last Month")
plt.legend()
plt.title("Actual vs Predicted Monthly Sales for SalesPerson 254")
plt.xlabel("Date")
plt.ylabel("Sales")
plt.xticks(rotation=45)
plt.grid()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

data = pd.read_csv('2023-2024-Horana-cleaned.csv')
data['DATE'] = pd.to_datetime(data['DATE'])
data['NetAmount'] = data['NetAmount'].abs()  # Ensure all values are positive

sales_265 = data[data['SalesPersonCode'] == '265']

sales_265_monthly = sales_265.resample('M', on='DATE')['NetAmount'].sum().reset_index()
sales_265_monthly.set_index('DATE', inplace=True)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_sales = scaler.fit_transform(sales_265_monthly[['NetAmount']])

def create_dataset(data, time_step=12):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:i + time_step, 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

time_step = 12  
X, y = create_dataset(scaled_sales, time_step)

X = X.reshape(X.shape[0], X.shape[1], 1)

train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

print(X_test[0:5])
print(y_test[0:5])

model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(time_step, 1)),
    LSTM(50, return_sequences=False),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, y_train, epochs=50, batch_size=8, verbose=1)

last_12_months = scaled_sales[-time_step:].reshape(1, time_step, 1)
predicted_scaled = model.predict(last_12_months)

predicted_sales = scaler.inverse_transform([[predicted_scaled[0][0]]])[0][0]

actual_sales = sales_265_monthly['NetAmount'].iloc[-1]

print(f"Actual Sales for Last Month: {actual_sales:.2f}")
print(f"Predicted Sales for Last Month: {predicted_sales:.2f}")

plt.figure(figsize=(10, 5))
plt.plot(sales_265_monthly.index[-12:], sales_265_monthly['NetAmount'][-12:], label="Actual", marker='o')
plt.axhline(y=predicted_sales, color='r', linestyle='--', label="Predicted Last Month")
plt.legend()
plt.title("Actual vs Predicted Monthly Sales for SalesPerson 254")
plt.xlabel("Date")
plt.ylabel("Sales")
plt.xticks(rotation=45)
plt.grid()
plt.show()

In [ ]:
import shap
import numpy as np

print("SHAP Version:", shap.__version__)

try:
    explainer = shap.DeepExplainer(model, X_train)
    shap_values = explainer.shap_values(X_test)
except Exception as e:
    print("DeepExplainer failed, switching to KernelExplainer...")
    background_samples = min(X_train.shape[0], 50)  # Avoid selecting more than available
    background = X_train[np.random.choice(X_train.shape[0], background_samples, replace=False)]
    explainer = shap.KernelExplainer(model.predict, background)

    X_test_2d = X_test.reshape(X_test.shape[0], -1)  # Flatten timesteps and features into a single dimension
    shap_values = explainer.shap_values(X_test_2d[:10])  # Use a small test set

print("\nSHAP Debugging Info")
print("SHAP Values Type:", type(shap_values))
print("SHAP Values Shape:", np.array(shap_values).shape)
print("X_test Shape:", X_test.shape)

try:
    expected_value = explainer.expected_value
    print("Expected Value:", expected_value)
except AttributeError:
    print("Error: 'expected_value' not found in explainer.")
    expected_value = None

print("\nSample Check:")
print("SHAP Values First Sample:", shap_values[0])
print("X_test First Sample:", X_test[0])

if shap_values.shape == X_test_2d.shape:
    shap.initjs()
    shap.force_plot(expected_value, shap_values[0], X_test_2d[0], feature_names=["NetAmount"])
    shap.summary_plot(shap_values, X_test_2d, feature_names=["NetAmount"])
    shap.dependence_plot(0, shap_values, X_test_2d, feature_names=["NetAmount"])
else:
    print("\nERROR: Shapes still do not match. Debug needed.")


In [ ]:
!pip install tensorflow==2.4.0

In [ ]:
import tensorflow as tf
import shap

print(tf.__version__)
print(shap.__version__)

print(X_train.shape) 
print(X_test.shape)   

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import shap

data = pd.read_csv('2023-2024-Horana-cleaned.csv')
data['DATE'] = pd.to_datetime(data['DATE'])
data['NetAmount'] = data['NetAmount'].abs()  # Ensure all values are positive

sales_265 = data[data['SalesPersonCode'] == '0']

sales_265_monthly = sales_265.resample('M', on='DATE')['NetAmount'].sum().reset_index()
sales_265_monthly.set_index('DATE', inplace=True)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_sales = scaler.fit_transform(sales_265_monthly[['NetAmount']])

def create_dataset(data, time_step=12):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:i + time_step, 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

time_step = 12  
X, y = create_dataset(scaled_sales, time_step)

if len(X) == 0 or len(y) == 0:
    raise ValueError("Not enough data to create sequences. Reduce time_step or use a larger dataset.")

X = X.reshape(X.shape[0], X.shape[1], 1)

train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

if len(X_train) == 0 or len(X_test) == 0:
    raise ValueError("Not enough data to split into train and test sets. Reduce time_step or use a larger dataset.")

print(f"X shape: {X.shape}")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(time_step, 1)),
    LSTM(50, return_sequences=False),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, y_train, epochs=50, batch_size=8, verbose=1)

last_12_months = scaled_sales[-time_step:].reshape(1, time_step, 1)
predicted_scaled = model.predict(last_12_months)

predicted_sales = scaler.inverse_transform([[predicted_scaled[0][0]]])[0][0]

actual_sales = sales_265_monthly['NetAmount'].iloc[-1]

print(f"Actual Sales for Last Month: {actual_sales:.2f}")
print(f"Predicted Sales for Last Month: {predicted_sales:.2f}")

plt.figure(figsize=(10, 5))
plt.plot(sales_265_monthly.index[-12:], sales_265_monthly['NetAmount'][-12:], label="Actual", marker='o')
plt.axhline(y=predicted_sales, color='r', linestyle='--', label="Predicted Last Month")
plt.legend()
plt.title("Actual vs Predicted Monthly Sales for SalesPerson 254")
plt.xlabel("Date")
plt.ylabel("Sales")
plt.xticks(rotation=45)
plt.grid()
plt.show()

background_data = X_train[:100]  
test_data = X_train[100:150]    

if len(test_data) == 0:
    raise ValueError("test_data is empty. Check the data splitting process.")

explainer = shap.GradientExplainer(model, background_data)

shap_values = explainer.shap_values(test_data)

shap.initjs()

shap.summary_plot(shap_values, test_data, feature_names=["Sales"], plot_type="bar")

sample_index = 0  
shap.force_plot(explainer.expected_value[0], shap_values[0][sample_index], test_data[sample_index], feature_names=["Sales"])

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import shap

data = pd.read_csv('2023-2024-Horana-cleaned.csv')
data['DATE'] = pd.to_datetime(data['DATE'])
data['NetAmount'] = data['NetAmount'].abs()  # Ensure all values are positive

sales_265 = data[data['SalesPersonCode'] == '265']

sales_265_monthly = sales_265.resample('M', on='DATE')['NetAmount'].sum().reset_index()
sales_265_monthly.set_index('DATE', inplace=True)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_sales = scaler.fit_transform(sales_265_monthly[['NetAmount']])

def create_dataset(data, time_step=12):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:i + time_step, 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

time_step = 12  
X, y = create_dataset(scaled_sales, time_step)

X = X.reshape(X.shape[0], X.shape[1], 1)

train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

print(X_test[0:5])
print(y_test[0:5])

model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(time_step, 1)),
    LSTM(50, return_sequences=False),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, y_train, epochs=50, batch_size=8, verbose=1)

last_12_months = scaled_sales[-time_step:].reshape(1, time_step, 1)
predicted_scaled = model.predict(last_12_months)

predicted_sales = scaler.inverse_transform([[predicted_scaled[0][0]]])[0][0]

actual_sales = sales_265_monthly['NetAmount'].iloc[-1]

print(f"Actual Sales for Last Month: {actual_sales:.2f}")
print(f"Predicted Sales for Last Month: {predicted_sales:.2f}")

plt.figure(figsize=(10, 5))
plt.plot(sales_265_monthly.index[-12:], sales_265_monthly['NetAmount'][-12:], label="Actual", marker='o')
plt.axhline(y=predicted_sales, color='r', linestyle='--', label="Predicted Last Month")
plt.legend()
plt.title("Actual vs Predicted Monthly Sales for SalesPerson 265")
plt.xlabel("Date")
plt.ylabel("Sales")
plt.xticks(rotation=45)
plt.grid()
plt.show()

try:
    background_samples_count = min(50, X_train.shape[0])  # Take 50 samples, or all if less than 50
    
    background_samples = X_train[np.random.choice(X_train.shape[0], background_samples_count, replace=False)]
    
    explainer = shap.KernelExplainer(model.predict, background_samples)
    
    X_test_2d = X_test.reshape(X_test.shape[0], -1)
    
    shap_values = explainer.shap_values(X_test_2d[:10])  # Use a small test set
    
    print("\n==== SHAP Debugging Info ====")
    print("SHAP Values Type:", type(shap_values))
    print("SHAP Values Shape:", np.array(shap_values).shape)
    print("X_test Shape:", X_test.shape)
    
    shap.initjs()
    shap.force_plot(explainer.expected_value[0], shap_values[0], X_test_2d[0], feature_names=[f"Month {i+1}" for i in range(time_step)])


except Exception as e:
    print(f"Error with SHAP: {e}")


In [ ]:
# !pip install optuna
import shap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import optuna
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

data = pd.read_csv('2023-2024-Horana-cleaned.csv')
data['DATE'] = pd.to_datetime(data['DATE'])
data['NetAmount'] = data['NetAmount'].abs()

sales_265 = data[data['SalesPersonCode'] == '265']

sales_265_monthly = sales_265.resample('ME', on='DATE')['NetAmount'].sum().reset_index()
sales_265_monthly.set_index('DATE', inplace=True)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_sales = scaler.fit_transform(sales_265_monthly[['NetAmount']])

def create_dataset(data, time_step=12):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:i + time_step, 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

time_step = 12  # 12 months for yearly trends
X, y = create_dataset(scaled_sales, time_step)
X = X.reshape(X.shape[0], X.shape[1], 1)

train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

def objective(trial):
    """Objective function for Optuna hyperparameter tuning."""
    n_layers = trial.suggest_int('n_layers', 1, 2)
    n_units = trial.suggest_int('n_units', 20, 100, step=10)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical('batch_size', [8, 16, 32])
    
    model = Sequential()
    model.add(LSTM(n_units, return_sequences=(n_layers == 2), input_shape=(time_step, 1)))
    model.add(Dropout(dropout_rate))
    
    if n_layers == 2:
        model.add(LSTM(n_units))
        model.add(Dropout(dropout_rate))
    
    model.add(Dense(1))
    
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')
    
    early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=50, batch_size=batch_size, verbose=0, callbacks=[early_stopping])
    
    loss = model.evaluate(X_test, y_test, verbose=0)
    return loss

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

best_params = study.best_params
print("Best hyperparameters:", best_params)

final_model = Sequential()
final_model.add(LSTM(best_params['n_units'], return_sequences=(best_params['n_layers'] == 2), input_shape=(time_step, 1)))
final_model.add(Dropout(best_params['dropout_rate']))

if best_params['n_layers'] == 2:
    final_model.add(LSTM(best_params['n_units']))
    final_model.add(Dropout(best_params['dropout_rate']))

final_model.add(Dense(1))
final_model.compile(optimizer=Adam(learning_rate=best_params['learning_rate']), loss='mean_squared_error')

final_model.fit(X_train, y_train, epochs=50, batch_size=best_params['batch_size'], verbose=1, callbacks=[EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)])

last_12_months = scaled_sales[-time_step:].reshape(1, time_step, 1)
predicted_scaled = final_model.predict(last_12_months)
predicted_sales = scaler.inverse_transform([[predicted_scaled[0][0]]])[0][0]

actual_sales = sales_265_monthly['NetAmount'].iloc[-1]

print(f"Actual Sales for Last Month: {actual_sales:.2f}")
print(f"Predicted Sales for Last Month: {predicted_sales:.2f}")

plt.figure(figsize=(10, 5))
plt.plot(sales_265_monthly.index[-12:], sales_265_monthly['NetAmount'][-12:], label="Actual", marker='o')
plt.axhline(y=predicted_sales, color='r', linestyle='--', label="Predicted Last Month")
plt.legend()
plt.title("Actual vs Predicted Monthly Sales for SalesPerson 265")
plt.xlabel("Date")
plt.ylabel("Sales")
plt.xticks(rotation=45)
plt.grid()
plt.show()


try:
    background_samples = X_train[np.random.choice(X_train.shape[0], min(50, X_train.shape[0]), replace=False)]

    explainer = shap.GradientExplainer(final_model, background_samples)

    X_test_shap = X_test[:3]  
    print(f'>>>>>>>>>>:{X_test_shap}')

    shap_values = explainer.shap_values(X_test_shap)

    shap_values_reshaped = np.array(shap_values).reshape(3, 12)  

    influential_reasons = []

    for i in range(shap_values_reshaped.shape[0]):  
        # print(f"\n==== top 3 influential reasons for prediction (sample {i+1}) ====")

        top_indices = np.argsort(np.abs(shap_values_reshaped[i]))[-3:][::-1]  

        for rank, j in enumerate(top_indices, 1):  
            contribution = shap_values_reshaped[i, j]
            direction = "increased" if contribution > 0 else "decreased"
            # message = f"month {j+1} {direction} prediction by {abs(contribution):.4f}"
            # print(f"top {rank}: {message}")

            influential_reasons.append((j+1, contribution))

    top_overall = sorted(influential_reasons, key=lambda x: abs(x[1]), reverse=True)[:3]

    print("\n==== highest 3 influential reasons overall ====")
    for rank, (month, value) in enumerate(top_overall, 1):
        direction = "increased" if value > 0 else "decreased"
        print(f"top {rank}: month {month} {direction} prediction by {abs(value):.4f}")

except Exception as e:
    print(f"error with shap: {e}")


In [ ]:
!pip install statsmodels

In [ ]:
import shap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import optuna
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

data = pd.read_csv('2023-2024-Horana-cleaned.csv')
data['DATE'] = pd.to_datetime(data['DATE'])
data['NetAmount'] = data['NetAmount'].abs()

sales_265 = data[data['SalesPersonCode'] == '265']

sales_265_monthly = sales_265.resample('ME', on='DATE')['NetAmount'].sum().reset_index()
sales_265_monthly.set_index('DATE', inplace=True)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_sales = scaler.fit_transform(sales_265_monthly[['NetAmount']])

def create_dataset(data, time_step=12):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:i + time_step, 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

time_step = 12 
X, y = create_dataset(scaled_sales, time_step)
X = X.reshape(X.shape[0], X.shape[1], 1)

train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

def objective(trial):
    """Objective function for Optuna hyperparameter tuning."""
    # Hyperparameter search space
    n_layers = trial.suggest_int('n_layers', 1, 2)
    n_units = trial.suggest_int('n_units', 20, 100, step=10)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical('batch_size', [8, 16, 32])
    
    model = Sequential()
    model.add(LSTM(n_units, return_sequences=(n_layers == 2), input_shape=(time_step, 1)))
    model.add(Dropout(dropout_rate))
    
    if n_layers == 2:
        model.add(LSTM(n_units))
        model.add(Dropout(dropout_rate))
    
    model.add(Dense(1))
    
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')
    
    early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=50, batch_size=batch_size, verbose=0, callbacks=[early_stopping])
    
    loss = model.evaluate(X_test, y_test, verbose=0)
    return loss

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

best_params = study.best_params
print("Best hyperparameters:", best_params)

final_model = Sequential()
final_model.add(LSTM(best_params['n_units'], return_sequences=(best_params['n_layers'] == 2), input_shape=(time_step, 1)))
final_model.add(Dropout(best_params['dropout_rate']))

if best_params['n_layers'] == 2:
    final_model.add(LSTM(best_params['n_units']))
    final_model.add(Dropout(best_params['dropout_rate']))

final_model.add(Dense(1))
final_model.compile(optimizer=Adam(learning_rate=best_params['learning_rate']), loss='mean_squared_error')

final_model.fit(X_train, y_train, epochs=50, batch_size=best_params['batch_size'], verbose=1, callbacks=[EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)])

last_12_months = scaled_sales[-time_step:].reshape(1, time_step, 1)
predicted_scaled = final_model.predict(last_12_months)
predicted_sales = scaler.inverse_transform([[predicted_scaled[0][0]]])[0][0]

next_month = sales_265_monthly.index[-1] + pd.DateOffset(months=1)

217print(f"predicted sales for {next_month.strftime('%b %Y')}: {predicted_sales:.2f}")

# plot results
plt.figure(figsize=(10, 5))
plt.plot(sales_265_monthly.index[-12:], sales_265_monthly['NetAmount'][-12:], label="actual", marker='o')
plt.scatter(next_month, predicted_sales, color='red', marker='x', label="predicted next month")
plt.legend()
plt.title("actual vs predicted monthly sales for salesperson 265")
plt.xlabel("date")
plt.ylabel("sales")
plt.xticks(rotation=45)
plt.grid()
plt.show()



try:
    background_samples = X_train[np.random.choice(X_train.shape[0], min(50, X_train.shape[0]), replace=False)]

    explainer = shap.GradientExplainer(final_model, background_samples)

    X_test_shap = X_test[:3]  
    # print(f'>>>>>>>>>>:{X_test_shap}')

    shap_values = explainer.shap_values(X_test_shap)

    shap_values_reshaped = np.array(shap_values).reshape(3, 12)  

    influential_reasons = []

    for i in range(shap_values_reshaped.shape[0]):  

        top_indices = np.argsort(np.abs(shap_values_reshaped[i]))[-3:][::-1]  

        for rank, j in enumerate(top_indices, 1):  
            contribution = shap_values_reshaped[i, j]
            direction = "increased" if contribution > 0 else "decreased"

            influential_reasons.append((j+1, contribution))

    top_overall = sorted(influential_reasons, key=lambda x: abs(x[1]), reverse=True)[:3]

    print("\n==== highest 3 influential reasons overall ====")
    for rank, (month, value) in enumerate(top_overall, 1):
        direction = "increased" if value > 0 else "decreased"
        print(f"top {rank}: month {month} {direction} prediction by {abs(value):.4f}")

except Exception as e:
    print(f"error with shap: {e}")

In [ ]:
import shap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import optuna
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

data = pd.read_csv('2023-2024-Horana-cleaned.csv')
data['DATE'] = pd.to_datetime(data['DATE'])
data['NetAmount'] = data['NetAmount'].abs()

kpi_data = pd.read_csv('KPI.csv')
kpi_data['Year-Month'] = pd.to_datetime(kpi_data['Year-Month'], format='%Y-%m')
kpi_data['Person Code'] = kpi_data['Person Code'].astype(str)

sales_265 = data[data['SalesPersonCode'] == '265']
sales_265_monthly = sales_265.resample('ME', on='DATE')['NetAmount'].sum().reset_index()
sales_265_monthly['Year-Month'] = sales_265_monthly['DATE'].dt.to_period('M').dt.to_timestamp()

merged_data = sales_265_monthly.merge(kpi_data, on=['Year-Month'], how='left')

features = ['NetAmount', 'Punctuality', 'Absenteeism', 'Maintaining Env', 'Sales Perf', 'Cust. Sat']
merged_data[features] = merged_data[features].apply(pd.to_numeric)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(merged_data[features])

def create_dataset(data, time_step=12):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:i + time_step])
        y.append(data[i + time_step, 0])  # NetAmount as target
    return np.array(X), np.array(y)

time_step = 12
X, y = create_dataset(scaled_data, time_step)
X = X.reshape(X.shape[0], X.shape[1], X.shape[2])

train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

def objective(trial):
    n_layers = trial.suggest_int('n_layers', 1, 2)
    n_units = trial.suggest_int('n_units', 20, 100, step=10)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical('batch_size', [8, 16, 32])
    
    model = Sequential()
    model.add(LSTM(n_units, return_sequences=(n_layers == 2), input_shape=(time_step, X.shape[2])))
    model.add(Dropout(dropout_rate))
    
    if n_layers == 2:
        model.add(LSTM(n_units))
        model.add(Dropout(dropout_rate))
    
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')
    
    early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=50, batch_size=batch_size, verbose=0, callbacks=[early_stopping])
    
    loss = model.evaluate(X_test, y_test, verbose=0)
    return loss

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

best_params = study.best_params
print("Best hyperparameters:", best_params)

final_model = Sequential()
final_model.add(LSTM(best_params['n_units'], return_sequences=(best_params['n_layers'] == 2), input_shape=(time_step, X.shape[2])))
final_model.add(Dropout(best_params['dropout_rate']))

if best_params['n_layers'] == 2:
    final_model.add(LSTM(best_params['n_units']))
    final_model.add(Dropout(best_params['dropout_rate']))

final_model.add(Dense(1))
final_model.compile(optimizer=Adam(learning_rate=best_params['learning_rate']), loss='mean_squared_error')

final_model.fit(X_train, y_train, epochs=50, batch_size=best_params['batch_size'], verbose=1, callbacks=[EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)])

last_12_months = scaled_data[-time_step:].reshape(1, time_step, scaled_data.shape[1])
predicted_scaled = final_model.predict(last_12_months)
predicted_sales = scaler.inverse_transform([[predicted_scaled[0][0]] + [0] * (scaled_data.shape[1] - 1)])[0][0]

next_month = merged_data['Year-Month'].iloc[-1] + pd.DateOffset(months=1)
print(f"Predicted sales for {next_month.strftime('%b %Y')}: {predicted_sales:.2f}")

try:
    background_samples = X_train[np.random.choice(X_train.shape[0], min(50, X_train.shape[0]), replace=False)]
    explainer = shap.GradientExplainer(final_model, background_samples)
    X_test_shap = X_test[:3]
    shap_values = explainer.shap_values(X_test_shap)
    shap_values_reshaped = np.array(shap_values).reshape(3, time_step, X.shape[2])
    influential_reasons = {}
    for i in range(shap_values_reshaped.shape[0]):
        for j in range(1, X.shape[2]):  
            month_index = np.argmax(np.abs(shap_values_reshaped[i, :, j]))
            month = merged_data['Year-Month'].iloc[train_size + month_index]
            key = (month, features[j])
            if key not in influential_reasons or abs(shap_values_reshaped[i, month_index, j]) > abs(influential_reasons[key]):
                influential_reasons[key] = shap_values_reshaped[i, month_index, j]
    top_overall = sorted(influential_reasons.items(), key=lambda x: abs(x[1]), reverse=True)[:3]
    print("\n==== Highest 3 Influential KPI Reasons Overall ====")
    for rank, ((month, kpi), value) in enumerate(top_overall, 1):
        direction = "increased" if value > 0 else "decreased"
        print(f"Top {rank}: ({month.strftime('%Y-%m')}) {kpi} {direction} prediction by {abs(value):.4f}")
except Exception as e:
    print(f"Error with SHAP: {e}")
